In [ ]:
import pandas as pd
import networkx as nx
import openpyxl
import distance
import matplotlib.pyplot as plt
import gseapy as gp
from gseapy import Biomart
bm = Biomart()
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import gseapy as gp
import numpy as np  
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import StandardScaler



# Kegg


In [ ]:

# Funzione per calcolare le cardinalità e le intersezioni
def card_and_int_disease_disease(original_genes, proposed_genes):
    original_set = set(original_genes)
    proposed_set = set(proposed_genes)
    cardinality_original = len(original_set)
    cardinality_proposed = len(proposed_set)
    intersection_set = original_set.intersection(proposed_set)
    cardinality_intersection = len(intersection_set)
    intersection_over_min_ratio = cardinality_intersection / min(cardinality_original, cardinality_proposed) if min(cardinality_original, cardinality_proposed) != 0 else 0
    
    
    return pd.Series([cardinality_original, cardinality_proposed, list(intersection_set), cardinality_intersection, intersection_over_min_ratio, list(original_set), list(proposed_set)])

# Funzione per l'analisi di arricchimento
def enrichr_analysis(gene_list, gene_sets, organism='Human'):
    enr = gp.enrichr(gene_list=gene_list, gene_sets=gene_sets, organism=organism)
    enriched_terms = enr.results[enr.results['Adjusted P-value'] < 0.05]
    terms_genes = {}
    for _, row in enriched_terms.iterrows():
        terms_genes[row['Term']] = row['Genes'].split(';')
    return list(terms_genes.keys()), terms_genes

# Funzione per calcolare le intersezioni
def calculate_intersections(terms1, terms2, genes1, genes2):
    set1 = set(terms1)
    set2 = set(terms2)
    intersection_terms = set1.intersection(set2)
    min_cardinality_terms = min(len(set1), len(set2))
    intersection_over_min_terms = len(intersection_terms) / min_cardinality_terms if min_cardinality_terms != 0 else 0
    
    intersection_genes = set()
    for term in intersection_terms:
        genes_intersection = set(genes1[term]).intersection(set(genes2[term]))
        intersection_genes.update(genes_intersection)
    
    return len(intersection_terms), intersection_terms, intersection_over_min_terms, list(intersection_genes)

# Set di dati per EnrichR
gene_sets = ['KEGG_2021_Human']
# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)

# Lista dei geni original disease (OD) e repurposed disease (RD)
od_genes = disease_disease_df['genes OD'].apply(lambda x: x.split(', ')).tolist()
rd_genes = disease_disease_df['genes RD'].apply(lambda x: x.split(', ')).tolist()

# Analisi di arricchimento per KEGG_2021_Human
terms_od, genes_od = enrichr_analysis([gene for sublist in od_genes for gene in sublist], gene_sets)
terms_rd, genes_rd = enrichr_analysis([gene for sublist in rd_genes for gene in sublist], gene_sets)

# Creazione del DataFrame per i risultati di KEGG_2021_Human
results_intersections_kegg = []

# Calcolo delle intersezioni per ogni coppia di malattie
for idx, row in disease_disease_df.iterrows():
    genes_od = row['genes OD'].split(', ')
    genes_rd = row['genes RD'].split(', ')
    
    terms_od, genes_od_dict = enrichr_analysis(genes_od, gene_sets)
    terms_rd, genes_rd_dict = enrichr_analysis(genes_rd, gene_sets)
    
    inter_od_rd, inter_od_rd_terms, iom_od_rd, inter_od_rd_genes = calculate_intersections(terms_od, terms_rd, genes_od_dict, genes_rd_dict)
    
    results_intersections_kegg.append({
        'Disease - Disease': row['Disease - Disease'],
        'OD ∩ RD': inter_od_rd,
        'Intersection over min % OD ∩ RD': iom_od_rd * 100
    })

# Convert to DataFrame
enrichment_results_kegg = pd.DataFrame(results_intersections_kegg)

# Save the results to an Excel file
output_file_path = "/Users/elisa/Desktop/Tesi/PROGETTO/Risultati_clustering/KEGG_2021_Human_Disease_Disease_Intersections.xlsx"
enrichment_results_kegg.to_excel(output_file_path, index=False)

# Visualizza il DataFrame
enrichment_results_kegg

In [ ]:
import pandas as pd
import gseapy as gp

# Funzione per calcolare le cardinalità e le intersezioni
def card_and_int_disease_disease(original_genes, proposed_genes):
    original_set = set(original_genes)
    proposed_set = set(proposed_genes)
    cardinality_original = len(original_set)
    cardinality_proposed = len(proposed_set)
    intersection_set = original_set.intersection(proposed_set)
    cardinality_intersection = len(intersection_set)
    intersection_over_min_ratio = cardinality_intersection / min(cardinality_original, cardinality_proposed) if min(cardinality_original, cardinality_proposed) != 0 else 0
    
    return pd.Series([cardinality_original, cardinality_proposed, list(intersection_set), cardinality_intersection, intersection_over_min_ratio, list(original_set), list(proposed_set)])

# Funzione per l'analisi di arricchimento
def enrichr_analysis(gene_list, gene_sets, organism='Human'):
    enr = gp.enrichr(gene_list=gene_list, gene_sets=gene_sets, organism=organism)
    enriched_terms = enr.results[enr.results['Adjusted P-value'] < 0.05]
    terms_genes = {}
    for _, row in enriched_terms.iterrows():
        terms_genes[row['Term']] = row['Genes'].split(';')
    return list(terms_genes.keys()), terms_genes

# Funzione per calcolare le intersezioni
def calculate_intersections(terms1, terms2, genes1, genes2):
    set1 = set(terms1)
    set2 = set(terms2)
    intersection_terms = set1.intersection(set2)
    min_cardinality_terms = min(len(set1), len(set2))
    intersection_over_min_terms = len(intersection_terms) / min_cardinality_terms if min_cardinality_terms != 0 else 0
    
    intersection_genes = set()
    for term in intersection_terms:
        genes_intersection = set(genes1[term]).intersection(set(genes2[term]))
        intersection_genes.update(genes_intersection)
    
    return len(intersection_terms), intersection_terms, intersection_over_min_terms, list(intersection_genes)

# Set di dati per EnrichR
gene_sets = ['KEGG_2021_Human']

# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)

# Lista dei geni original disease (OD) e repurposed disease (RD)
od_genes = disease_disease_df['genes OD'].apply(lambda x: x.split(', ')).tolist()
rd_genes = disease_disease_df['genes RD'].apply(lambda x: x.split(', ')).tolist()

# Creazione del DataFrame per i risultati di KEGG_2021_Human
results_intersections_kegg = []

# Calcolo delle intersezioni per ogni coppia di malattie
for idx, row in disease_disease_df.iterrows():
    genes_od = row['genes OD'].split(', ')
    genes_rd = row['genes RD'].split(', ')
    
    terms_od, genes_od_dict = enrichr_analysis(genes_od, gene_sets)
    terms_rd, genes_rd_dict = enrichr_analysis(genes_rd, gene_sets)
    
    inter_od_rd, inter_od_rd_terms, iom_od_rd, inter_od_rd_genes = calculate_intersections(terms_od, terms_rd, genes_od_dict, genes_rd_dict)
    
    results_intersections_kegg.append({
        'Disease - Disease': row['Disease - Disease'],
        'OD ∩ RD': inter_od_rd,
        'Intersection Terms OD ∩ RD': list(inter_od_rd_terms),
        'Intersection Genes OD ∩ RD': list(inter_od_rd_genes),
        'Intersection over min % OD ∩ RD': iom_od_rd * 100
    })

# Convert to DataFrame
enrichment_results_kegg = pd.DataFrame(results_intersections_kegg)

# Save the results to an Excel file
#output_file_path = "/Users/elisa/Desktop/Tesi/PROGETTO/Risultati_clustering/KEGG_2021_Human_Disease_Disease_Intersections.xlsx"
#enrichment_results_kegg.to_excel(output_file_path, index=False)

# Visualizza il DataFrame
enrichment_results_kegg


## Reactome

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import gseapy as gp

# Funzione per calcolare le cardinalità e le intersezioni
def card_and_int_disease_disease(original_genes, proposed_genes):
    original_set = set(original_genes)
    proposed_set = set(proposed_genes)
    cardinality_original = len(original_set)
    cardinality_proposed = len(proposed_set)
    intersection_set = original_set.intersection(proposed_set)
    cardinality_intersection = len(intersection_set)
    intersection_over_min_ratio = cardinality_intersection / min(cardinality_original, cardinality_proposed) if min(cardinality_original, cardinality_proposed) != 0 else 0
    
    return pd.Series([cardinality_original, cardinality_proposed, list(intersection_set), cardinality_intersection, intersection_over_min_ratio, list(original_set), list(proposed_set)])

# Funzione per l'analisi di arricchimento
def enrichr_analysis(gene_list, gene_sets, organism='Human'):
    enr = gp.enrichr(gene_list=gene_list, gene_sets=gene_sets, organism=organism)
    enriched_terms = enr.results[enr.results['Adjusted P-value'] < 0.05]
    terms_genes = {}
    for _, row in enriched_terms.iterrows():
        terms_genes[row['Term']] = row['Genes'].split(';')
    return list(terms_genes.keys()), terms_genes

# Funzione per calcolare le intersezioni
def calculate_intersections(terms1, terms2, genes1, genes2):
    set1 = set(terms1)
    set2 = set(terms2)
    intersection_terms = set1.intersection(set2)
    min_cardinality_terms = min(len(set1), len(set2))
    intersection_over_min_terms = len(intersection_terms) / min_cardinality_terms if min_cardinality_terms != 0 else 0
    
    intersection_genes = set()
    for term in intersection_terms:
        genes_intersection = set(genes1[term]).intersection(set(genes2[term]))
        intersection_genes.update(genes_intersection)
    
    return len(intersection_terms), intersection_terms, intersection_over_min_terms, list(intersection_genes)

# Set di dati per EnrichR
gene_sets = ['Reactome_2022']

# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)


# Lista dei geni original disease (OD) e repurposed disease (RD)
od_genes = disease_disease_df['genes OD'].apply(lambda x: x.split(', ')).tolist()
rd_genes = disease_disease_df['genes RD'].apply(lambda x: x.split(', ')).tolist()

# Analisi di arricchimento per Reactome_2022
terms_od, genes_od = enrichr_analysis([gene for sublist in od_genes for gene in sublist], gene_sets)
terms_rd, genes_rd = enrichr_analysis([gene for sublist in rd_genes for gene in sublist], gene_sets)

# Creazione del DataFrame per i risultati di Reactome_2022
results_intersections_reactome = []

# Calcolo delle intersezioni per ogni coppia di malattie
for idx, row in disease_disease_df.iterrows():
    genes_od = row['genes OD'].split(', ')
    genes_rd = row['genes RD'].split(', ')
    
    terms_od, genes_od_dict = enrichr_analysis(genes_od, gene_sets)
    terms_rd, genes_rd_dict = enrichr_analysis(genes_rd, gene_sets)
    
    inter_od_rd, inter_od_rd_terms, iom_od_rd, inter_od_rd_genes = calculate_intersections(terms_od, terms_rd, genes_od_dict, genes_rd_dict)
    
    results_intersections_reactome.append({
        'Disease - Disease': row['Disease - Disease'],
        'OD ∩ RD': inter_od_rd,
        'Intersection over min % OD ∩ RD': iom_od_rd * 100
    })

# Convert to DataFrame
enrichment_results_reactome = pd.DataFrame(results_intersections_reactome)

# Save the results to an Excel file
output_file_path = "/Users/elisa/Desktop/Tesi/PROGETTO/Risultati_clustering/Reactome_2022_Disease_Disease_Intersections.xlsx"
enrichment_results_reactome.to_excel(output_file_path, index=False)

# Visualizza il DataFrame
enrichment_results_reactome

## Biological Process

In [ ]:
# Funzione per calcolare le cardinalità e le intersezioni
def card_and_int_disease_disease(original_genes, proposed_genes):
    original_set = set(original_genes)
    proposed_set = set(proposed_genes)
    cardinality_original = len(original_set)
    cardinality_proposed = len(proposed_set)
    intersection_set = original_set.intersection(proposed_set)
    cardinality_intersection = len(intersection_set)
    intersection_over_min_ratio = cardinality_intersection / min(cardinality_original, cardinality_proposed) if min(cardinality_original, cardinality_proposed) != 0 else 0
    
    
    return pd.Series([cardinality_original, cardinality_proposed, list(intersection_set), cardinality_intersection, intersection_over_min_ratio, list(original_set), list(proposed_set)])

# Funzione per l'analisi di arricchimento
def enrichr_analysis(gene_list, gene_sets, organism='Human'):
    enr = gp.enrichr(gene_list=gene_list, gene_sets=gene_sets, organism=organism)
    enriched_terms = enr.results[enr.results['Adjusted P-value'] < 0.05]
    terms_genes = {}
    for _, row in enriched_terms.iterrows():
        terms_genes[row['Term']] = row['Genes'].split(';')
    return list(terms_genes.keys()), terms_genes

# Funzione per calcolare le intersezioni
def calculate_intersections(terms1, terms2, genes1, genes2):
    set1 = set(terms1)
    set2 = set(terms2)
    intersection_terms = set1.intersection(set2)
    min_cardinality_terms = min(len(set1), len(set2))
    intersection_over_min_terms = len(intersection_terms) / min_cardinality_terms if min_cardinality_terms != 0 else 0
    
    intersection_genes = set()
    for term in intersection_terms:
        genes_intersection = set(genes1[term]).intersection(set(genes2[term]))
        intersection_genes.update(genes_intersection)
    
    return len(intersection_terms), intersection_terms, intersection_over_min_terms, list(intersection_genes)

# Set di dati per EnrichR
gene_sets = ['GO_Biological_Process_2021']

# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)



# Lista dei geni original disease (OD) e repurposed disease (RD)
od_genes = disease_disease_df['genes OD'].apply(lambda x: x.split(', ')).tolist()
rd_genes = disease_disease_df['genes RD'].apply(lambda x: x.split(', ')).tolist()

# Analisi di arricchimento per GO_Biological_Process_2021
terms_od, genes_od = enrichr_analysis([gene for sublist in od_genes for gene in sublist], gene_sets)
terms_rd, genes_rd = enrichr_analysis([gene for sublist in rd_genes for gene in sublist], gene_sets)

# Creazione del DataFrame per i risultati di GO_Biological_Process_2021
results_intersections_bp= []

# Calcolo delle intersezioni per ogni coppia di malattie
for idx, row in disease_disease_df.iterrows():
    genes_od = row['genes OD'].split(', ')
    genes_rd = row['genes RD'].split(', ')
    
    terms_od, genes_od_dict = enrichr_analysis(genes_od, gene_sets)
    terms_rd, genes_rd_dict = enrichr_analysis(genes_rd, gene_sets)
    
    inter_od_rd, inter_od_rd_terms, iom_od_rd, inter_od_rd_genes = calculate_intersections(terms_od, terms_rd, genes_od_dict, genes_rd_dict)
    
    results_intersections_bp.append({
        'Disease - Disease': row['Disease - Disease'],
        'OD ∩ RD': inter_od_rd,
        'Intersection over min % OD ∩ RD': iom_od_rd * 100
    })

# Convert to DataFrame
enrichment_results_bp = pd.DataFrame(results_intersections_bp)

# Save the results to an Excel file
output_file_path = "/Users/elisa/Desktop/Tesi/PROGETTO/Risultati_clustering/GO_Biological_Process_2021_Disease_Disease_Intersections.xlsx"
enrichment_results_bp.to_excel(output_file_path, index=False)

# Visualizza il DataFrame
enrichment_results_bp

## Molecular Function

In [ ]:
# Funzione per calcolare le cardinalità e le intersezioni
def card_and_int_disease_disease(original_genes, proposed_genes):
    original_set = set(original_genes)
    proposed_set = set(proposed_genes)
    cardinality_original = len(original_set)
    cardinality_proposed = len(proposed_set)
    intersection_set = original_set.intersection(proposed_set)
    cardinality_intersection = len(intersection_set)
    intersection_over_min_ratio = cardinality_intersection / min(cardinality_original, cardinality_proposed) if min(cardinality_original, cardinality_proposed) != 0 else 0
    
    
    return pd.Series([cardinality_original, cardinality_proposed, list(intersection_set), cardinality_intersection, intersection_over_min_ratio, list(original_set), list(proposed_set)])

# Funzione per l'analisi di arricchimento
def enrichr_analysis(gene_list, gene_sets, organism='Human'):
    enr = gp.enrichr(gene_list=gene_list, gene_sets=gene_sets, organism=organism)
    enriched_terms = enr.results[enr.results['Adjusted P-value'] < 0.05]
    terms_genes = {}
    for _, row in enriched_terms.iterrows():
        terms_genes[row['Term']] = row['Genes'].split(';')
    return list(terms_genes.keys()), terms_genes

# Funzione per calcolare le intersezioni
def calculate_intersections(terms1, terms2, genes1, genes2):
    set1 = set(terms1)
    set2 = set(terms2)
    intersection_terms = set1.intersection(set2)
    min_cardinality_terms = min(len(set1), len(set2))
    intersection_over_min_terms = len(intersection_terms) / min_cardinality_terms if min_cardinality_terms != 0 else 0
    
    intersection_genes = set()
    for term in intersection_terms:
        genes_intersection = set(genes1[term]).intersection(set(genes2[term]))
        intersection_genes.update(genes_intersection)
    
    return len(intersection_terms), intersection_terms, intersection_over_min_terms, list(intersection_genes)

# Set di dati per EnrichR
gene_sets = ['GO_Molecular_Function_2021']

# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)



# Lista dei geni original disease (OD) e repurposed disease (RD)
od_genes = disease_disease_df['genes OD'].apply(lambda x: x.split(', ')).tolist()
rd_genes = disease_disease_df['genes RD'].apply(lambda x: x.split(', ')).tolist()

# Analisi di arricchimento per GO_Biological_Process_2021
terms_od, genes_od = enrichr_analysis([gene for sublist in od_genes for gene in sublist], gene_sets)
terms_rd, genes_rd = enrichr_analysis([gene for sublist in rd_genes for gene in sublist], gene_sets)

# Creazione del DataFrame per i risultati di GO_Biological_Process_2021
results_intersections_mf= []

# Calcolo delle intersezioni per ogni coppia di malattie
for idx, row in disease_disease_df.iterrows():
    genes_od = row['genes OD'].split(', ')
    genes_rd = row['genes RD'].split(', ')
    
    terms_od, genes_od_dict = enrichr_analysis(genes_od, gene_sets)
    terms_rd, genes_rd_dict = enrichr_analysis(genes_rd, gene_sets)
    
    inter_od_rd, inter_od_rd_terms, iom_od_rd, inter_od_rd_genes = calculate_intersections(terms_od, terms_rd, genes_od_dict, genes_rd_dict)
    
    results_intersections_mf.append({
        'Disease - Disease': row['Disease - Disease'],
        'OD ∩ RD': inter_od_rd,
        'Intersection over min % OD ∩ RD': iom_od_rd * 100
    })

# Convert to DataFrame
enrichment_results_mf = pd.DataFrame(results_intersections_mf)

# Save the results to an Excel file
output_file_path = "/Users/elisa/Desktop/Tesi/PROGETTO/Risultati_clustering/GO_Molecular_Function_2021_Disease_Disease_Intersections.xlsx"
enrichment_results_mf.to_excel(output_file_path, index=False)

# Visualizza il DataFrame
enrichment_results_mf

,Disease - Disease,OD ∩ RD,Intersection over min % OD ∩ RD
0,Colorectal carcinoma - Human Immunodeficiency ...,12,22.222222
1,Colorectal carcinoma - Acquired Immunodeficien...,6,11.111111
2,Idiopathic intracranial hypertension - Alopecia,0,0.000000
3,Angina pectoris - Erectile dysfunction,13,38.235294
4,Nausea and vomiting - Erythema nodosum leprosum,4,20.000000
5,Nausea and vomiting - Multiple myeloma,3,30.000000
6,Generalized Osteoarthritis - Multiple adenomat...,1,6.250000
7,Rheumatoid arthritis - Multiple adenomatous po...,4,25.000000
8,Parkinson disease - Attention deficit hyperact...,3,10.714286
9,Depressive disorder - Urinary Stress Incontinence,30,48.387097


# Cellular Component

In [ ]:
# Funzione per calcolare le cardinalità e le intersezioni
def card_and_int_disease_disease(original_genes, proposed_genes):
    original_set = set(original_genes)
    proposed_set = set(proposed_genes)
    cardinality_original = len(original_set)
    cardinality_proposed = len(proposed_set)
    intersection_set = original_set.intersection(proposed_set)
    cardinality_intersection = len(intersection_set)
    intersection_over_min_ratio = cardinality_intersection / min(cardinality_original, cardinality_proposed) if min(cardinality_original, cardinality_proposed) != 0 else 0
    
    
    return pd.Series([cardinality_original, cardinality_proposed, list(intersection_set), cardinality_intersection, intersection_over_min_ratio, list(original_set), list(proposed_set)])

# Funzione per l'analisi di arricchimento
def enrichr_analysis(gene_list, gene_sets, organism='Human'):
    enr = gp.enrichr(gene_list=gene_list, gene_sets=gene_sets, organism=organism)
    enriched_terms = enr.results[enr.results['Adjusted P-value'] < 0.05]
    terms_genes = {}
    for _, row in enriched_terms.iterrows():
        terms_genes[row['Term']] = row['Genes'].split(';')
    return list(terms_genes.keys()), terms_genes

# Funzione per calcolare le intersezioni
def calculate_intersections(terms1, terms2, genes1, genes2):
    set1 = set(terms1)
    set2 = set(terms2)
    intersection_terms = set1.intersection(set2)
    min_cardinality_terms = min(len(set1), len(set2))
    intersection_over_min_terms = len(intersection_terms) / min_cardinality_terms if min_cardinality_terms != 0 else 0
    
    intersection_genes = set()
    for term in intersection_terms:
        genes_intersection = set(genes1[term]).intersection(set(genes2[term]))
        intersection_genes.update(genes_intersection)
    
    return len(intersection_terms), intersection_terms, intersection_over_min_terms, list(intersection_genes)

# Set di dati per EnrichR
gene_sets = ['GO_Cellular_Component_2021']

# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)


# Lista dei geni original disease (OD) e repurposed disease (RD)
od_genes = disease_disease_df['genes OD'].apply(lambda x: x.split(', ')).tolist()
rd_genes = disease_disease_df['genes RD'].apply(lambda x: x.split(', ')).tolist()

# Analisi di arricchimento per GO_Biological_Process_2021
terms_od, genes_od = enrichr_analysis([gene for sublist in od_genes for gene in sublist], gene_sets)
terms_rd, genes_rd = enrichr_analysis([gene for sublist in rd_genes for gene in sublist], gene_sets)

# Creazione del DataFrame per i risultati di GO_Biological_Process_2021
results_intersections_cc = []

# Calcolo delle intersezioni per ogni coppia di malattie
for idx, row in disease_disease_df.iterrows():
    genes_od = row['genes OD'].split(', ')
    genes_rd = row['genes RD'].split(', ')
    
    terms_od, genes_od_dict = enrichr_analysis(genes_od, gene_sets)
    terms_rd, genes_rd_dict = enrichr_analysis(genes_rd, gene_sets)
    
    inter_od_rd, inter_od_rd_terms, iom_od_rd, inter_od_rd_genes = calculate_intersections(terms_od, terms_rd, genes_od_dict, genes_rd_dict)
    
    results_intersections_cc.append({
        'Disease - Disease': row['Disease - Disease'],
        'OD ∩ RD': inter_od_rd,
        'Intersection over min % OD ∩ RD': iom_od_rd * 100
    })

# Convert to DataFrame
enrichment_results_cc = pd.DataFrame(results_intersections_cc)

# Save the results to an Excel file
output_file_path = "/Users/elisa/Desktop/Tesi/PROGETTO/Risultati_clustering/GO_Cellular_Component_Disease_Disease_Intersections.xlsx"
enrichment_results_cc.to_excel(output_file_path, index=False)

# Visualizza il DataFrame
enrichment_results_cc

ConnectionError: HTTPSConnectionPool(host='maayanlab.cloud', port=443): Max retries exceeded with url: /Enrichr/addList (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1681d3bb0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

# DisGenet

In [ ]:
import pandas as pd

# Funzione per calcolare le intersezioni dei geni
def calculate_gene_intersections(row):
    genes_od = set(row['genes OD'].split(', '))
    genes_rd = set(row['genes RD'].split(', '))
    intersection_genes = genes_od.intersection(genes_rd)
    intersection_size = len(intersection_genes)
    min_size = min(len(genes_od), len(genes_rd))
    intersection_over_min = intersection_size / min_size if min_size != 0 else 0
    return pd.Series([intersection_size, intersection_genes, intersection_over_min])

# Carica i dati dai file Excel
disease_disease_df = pd.read_excel("/Users/elisa/Desktop/Tesi/PROGETTO/Disease_Disease_summary.xlsx")

# Converto tutti i valori delle colonne dei geni in stringhe
disease_disease_df['genes OD'] = disease_disease_df['genes OD'].astype(str)
disease_disease_df['genes RD'] = disease_disease_df['genes RD'].astype(str)

# Calcola le intersezioni dei geni
disease_disease_df[['Intersection size', 'Intersection genes', 'Intersection over min %']] = disease_disease_df.apply(calculate_gene_intersections, axis=1)

# Visualizza il DataFrame aggiornato
disease_disease_df[['Disease - Disease', 'Intersection size', 'Intersection over min %']]
